In [55]:
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [56]:
import pandas as pd
import sys
sys.path.append('../modules')

from preprocess import preprocess_test_data, load_scaler

from tensorflow.keras.models import load_model 
def asymmetric_loss(RUL_true, RUL_predicted, a1=10, a2=13):
    diff = tf.subtract(RUL_true, RUL_predicted)
    loss = tf.reduce_sum(tf.where(diff < 0, tf.exp(-diff/a1)-1, tf.exp(diff/a2)-1), axis=0)
    return loss


In [57]:
test_data_path = "../dataset/test_FD001.txt"

In [58]:
test_data = pd.read_csv(test_data_path, sep="\s+", header=None)
test_data

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,1,1,0.0023,0.0003,100.0,518.67,643.02,1585.29,1398.21,14.62,...,521.72,2388.03,8125.55,8.4052,0.03,392,2388,100.0,38.86,23.3735
1,1,2,-0.0027,-0.0003,100.0,518.67,641.71,1588.45,1395.42,14.62,...,522.16,2388.06,8139.62,8.3803,0.03,393,2388,100.0,39.02,23.3916
2,1,3,0.0003,0.0001,100.0,518.67,642.46,1586.94,1401.34,14.62,...,521.97,2388.03,8130.10,8.4441,0.03,393,2388,100.0,39.08,23.4166
3,1,4,0.0042,0.0000,100.0,518.67,642.44,1584.12,1406.42,14.62,...,521.38,2388.05,8132.90,8.3917,0.03,391,2388,100.0,39.00,23.3737
4,1,5,0.0014,0.0000,100.0,518.67,642.51,1587.19,1401.92,14.62,...,522.15,2388.03,8129.54,8.4031,0.03,390,2388,100.0,38.99,23.4130
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13091,100,194,0.0049,0.0000,100.0,518.67,643.24,1599.45,1415.79,14.62,...,520.69,2388.00,8213.28,8.4715,0.03,394,2388,100.0,38.65,23.1974
13092,100,195,-0.0011,-0.0001,100.0,518.67,643.22,1595.69,1422.05,14.62,...,521.05,2388.09,8210.85,8.4512,0.03,395,2388,100.0,38.57,23.2771
13093,100,196,-0.0006,-0.0003,100.0,518.67,643.44,1593.15,1406.82,14.62,...,521.18,2388.04,8217.24,8.4569,0.03,395,2388,100.0,38.62,23.2051
13094,100,197,-0.0038,0.0001,100.0,518.67,643.26,1594.99,1419.36,14.62,...,521.33,2388.08,8220.48,8.4711,0.03,395,2388,100.0,38.66,23.2699


In [59]:
from __init__ import SCALER_PATH


In [60]:
preprocessed_data = preprocess_test_data(test_data, SCALER_PATH,30)

c:\Users\usatoleb\miniconda3\envs\action_project\lib\site-packages\sklearn\base.py:347: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.4.0 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
DEBUG:root:Scaled test DataFrame head: 
   oper_set_1  oper_set_2  temp_lpc_outlet  temp_hpc_outlet  temp_lpt_outlet  \
0    1.055599    1.015677         0.678077        -0.853550        -1.191480   
1   -1.230366   -1.031720        -1.941707        -0.338137        -1.501467   
2    0.141213    0.333211        -0.441831        -0.584426        -0.843717   
3    1.924266   -0.008022        -0.481827        -1.044384        -0.279297   
4    0.644125   -0.008022        -0.341839        -0.543650        -0.779276   

   px_hpc_outlet  phys_fan_speed  phys_core_speed  stat_px_

In [61]:
preprocessed_data

array([[[-1.23036553e+00, -1.03172035e+00, -1.94170729e+00, ...,
         -1.36017565e-01,  1.12718287e+00,  9.41305176e-01],
        [ 1.41213246e-01,  3.33211040e-01, -4.41830730e-01, ...,
         -1.36017565e-01,  1.45914768e+00,  1.17225579e+00],
        [ 1.92426566e+00, -8.02180852e-03, -4.81827438e-01, ...,
         -1.42740201e+00,  1.01652793e+00,  7.75944537e-01],
        ...,
        [ 6.44125465e-01,  3.33211040e-01, -1.46174679e+00, ...,
         -7.81709786e-01,  1.12718287e+00,  6.68783452e-01],
        [-1.13892695e+00,  1.35690959e+00,  2.18114956e-01, ...,
         -1.42740201e+00,  1.51447515e+00,  1.08264695e+00],
        [-2.70260387e-01,  1.35690959e+00, -2.01850481e-01, ...,
         -1.36017565e-01, -3.46939782e-02,  6.05041083e-01]],

       [[ 1.78710778e+00,  1.35690959e+00,  7.81264768e-02, ...,
         -1.36017565e-01, -2.56003854e-01, -3.71418110e-01],
        [-1.73327775e+00, -1.03172035e+00, -7.21807687e-01, ...,
          5.09674655e-01,  4.07925773e

In [62]:
model = load_model('../items/final_model.h5', custom_objects={'asymmetric_loss': asymmetric_loss})
RUL_predicted = model.predict(preprocessed_data)
print("Predicted RUL:", RUL_predicted)

4/4 [==============================] - 1s 11ms/step
Predicted RUL: [[109.63445  ]
 [121.536064 ]
 [ 54.281513 ]
 [ 60.84145  ]
 [ 98.5295   ]
 [100.612076 ]
 [ 94.00808  ]
 [ 94.54713  ]
 [121.13082  ]
 [ 72.60619  ]
 [ 87.53351  ]
 [ 93.085686 ]
 [ 65.72667  ]
 [117.13848  ]
 [ 92.44744  ]
 [ 90.503296 ]
 [ 52.25364  ]
 [ 23.87915  ]
 [ 73.54272  ]
 [ 16.158188 ]
 [ 60.026306 ]
 [126.141914 ]
 [120.60369  ]
 [ 19.010937 ]
 [112.4067   ]
 [115.45308  ]
 [ 87.57694  ]
 [107.7212   ]
 [106.49873  ]
 [107.99688  ]
 [  7.9710593]
 [ 48.813694 ]
 [111.08709  ]
 [  4.0314164]
 [  3.9967885]
 [ 18.234777 ]
 [ 22.77562  ]
 [ 53.099808 ]
 [127.29288  ]
 [ 32.458683 ]
 [ 21.405516 ]
 [  4.5304694]
 [ 59.175594 ]
 [ 92.07641  ]
 [ 75.10435  ]
 [ 43.492622 ]
 [119.49157  ]
 [110.070786 ]
 [ 16.282825 ]
 [ 82.771416 ]
 [104.69519  ]
 [ 30.419186 ]
 [ 28.027645 ]
 [118.474686 ]
 [119.33222  ]
 [  8.72989  ]
 [104.168625 ]
 [ 39.70265  ]
 [110.5586   ]
 [105.12835  ]
 [ 17.240353 ]
 [ 38.34861  ]
 [ 

## test window size +30

In [63]:
window_30_data = pd.read_csv('../input_data_windowmore30/engine_1.csv',header=None)

In [64]:
window_30_data

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,521.66,2388.02,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,522.28,2388.07,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,522.42,2388.03,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,522.86,2388.08,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,522.19,2388.04,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,1,188,-0.0067,0.0003,100.0,518.67,643.75,1602.38,1422.78,14.62,...,519.79,2388.23,8117.69,8.5207,0.03,396,2388,100.0,38.51,22.9588
188,1,189,-0.0006,0.0002,100.0,518.67,644.18,1596.17,1428.01,14.62,...,519.58,2388.33,8117.51,8.5183,0.03,395,2388,100.0,38.48,23.1127
189,1,190,-0.0027,0.0001,100.0,518.67,643.64,1599.22,1425.95,14.62,...,520.04,2388.35,8112.58,8.5223,0.03,398,2388,100.0,38.49,23.0675
190,1,191,-0.0000,-0.0004,100.0,518.67,643.34,1602.36,1425.77,14.62,...,519.57,2388.30,8114.61,8.5174,0.03,394,2388,100.0,38.45,23.1295


In [65]:
window_size = 30
preprocessed_data = preprocess_test_data(window_30_data, SCALER_PATH,window_size)

c:\Users\usatoleb\miniconda3\envs\action_project\lib\site-packages\sklearn\base.py:347: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.4.0 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
DEBUG:root:Scaled test DataFrame head: 
   oper_set_1  oper_set_2  temp_lpc_outlet  temp_hpc_outlet  temp_lpt_outlet  \
0   -0.315980   -1.372953        -1.721725        -0.134255        -0.925936   
1    0.872722   -1.031720        -1.061780         0.211528        -0.643726   
2   -1.961874    1.015677        -0.661813        -0.413166        -0.525953   
3    0.324090   -0.008022        -0.661813        -1.261314        -0.784831   
4   -0.864611   -0.690488        -0.621816        -1.251528        -0.301518   

   px_hpc_outlet  phys_fan_speed  phys_core_speed  stat_px_

In [66]:
preprocessed_data.shape

(1, 30, 15)

In [67]:
preprocessed_data

array([[[ 1.41213246e-01, -1.37295320e+00,  3.38105080e-01,
          1.63380849e+00,  1.35063290e+00, -1.08207299e+00,
          1.45594375e+00, -7.24693922e-01,  1.30609048e+00,
         -1.50971715e+00,  1.44398902e+00,  1.88657676e+00,
         -1.36017565e-01, -9.19933480e-01, -1.36542955e+00],
        [ 2.32651831e-01, -6.90487506e-01,  9.78052411e-01,
          1.37610180e+00,  8.69542456e-01, -1.65829807e+00,
          1.03331186e+00, -7.39185137e-01,  5.94695614e-01,
         -1.06228107e+00,  2.55637981e+00,  1.01733913e+00,
          5.09674655e-01, -1.08591589e+00, -1.04948911e+00],
        [ 4.61248295e-01,  1.35690959e+00,  1.58119893e-01,
          1.06130817e+00, -9.37494933e-02, -6.52728806e-01,
          1.31506645e+00, -1.18433715e+00,  1.30609048e+00,
         -9.80929059e-01,  2.00018441e+00,  1.33463753e+00,
         -1.36017565e-01, -1.25189829e+00, -7.09529808e-01],
        [-1.00176907e+00, -1.03172035e+00,  1.31802443e+00,
          1.01074547e+00,  1.49284901

In [68]:
model = load_model('../items/final_model.h5', custom_objects={'asymmetric_loss': asymmetric_loss})
RUL_predicted = model.predict(preprocessed_data)
print("Predicted RUL:", RUL_predicted)

1/1 [==============================] - 1s 759ms/step
Predicted RUL: [[1.5879439]]


## test window size 30

In [69]:
window_30_data = pd.read_csv('../input_data_window_30/engine_1_cycle_1_to_30.csv',header=None)

In [70]:
window_30_data

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,521.66,2388.02,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,522.28,2388.07,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,522.42,2388.03,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,522.86,2388.08,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,522.19,2388.04,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044
5,1,6,-0.0043,-0.0001,100.0,518.67,642.10,1584.47,1398.37,14.62,...,521.68,2388.03,8132.85,8.4108,0.03,391,2388,100.0,38.98,23.3669
6,1,7,0.0010,0.0001,100.0,518.67,642.48,1592.32,1397.77,14.62,...,522.32,2388.03,8132.32,8.3974,0.03,392,2388,100.0,39.10,23.3774
7,1,8,-0.0034,0.0003,100.0,518.67,642.56,1582.96,1400.97,14.62,...,522.47,2388.03,8131.07,8.4076,0.03,391,2388,100.0,38.97,23.3106
8,1,9,0.0008,0.0001,100.0,518.67,642.12,1590.98,1394.80,14.62,...,521.79,2388.05,8125.69,8.3728,0.03,392,2388,100.0,39.05,23.4066
9,1,10,-0.0033,0.0001,100.0,518.67,641.71,1591.24,1400.46,14.62,...,521.79,2388.06,8129.38,8.4286,0.03,393,2388,100.0,38.95,23.4694


In [71]:
window_size = 30  
preprocessed_data = preprocess_test_data(window_30_data, SCALER_PATH, window_size)
preprocessed_data.shape


c:\Users\usatoleb\miniconda3\envs\action_project\lib\site-packages\sklearn\base.py:347: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.4.0 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
DEBUG:root:Scaled test DataFrame head: 
   oper_set_1  oper_set_2  temp_lpc_outlet  temp_hpc_outlet  temp_lpt_outlet  \
0   -0.315980   -1.372953        -1.721725        -0.134255        -0.925936   
1    0.872722   -1.031720        -1.061780         0.211528        -0.643726   
2   -1.961874    1.015677        -0.661813        -0.413166        -0.525953   
3    0.324090   -0.008022        -0.661813        -1.261314        -0.784831   
4   -0.864611   -0.690488        -0.621816        -1.251528        -0.301518   

   px_hpc_outlet  phys_fan_speed  phys_core_speed  stat_px_

(1, 30, 15)

In [72]:
preprocessed_data

array([[[-0.31597968, -1.3729532 , -1.72172539, -0.13425518,
         -0.92593596,  1.12114057, -0.51633841, -0.86281332,
         -0.2664666 ,  0.33426184, -1.05889024, -0.6038157 ,
         -0.78170979,  1.34849274,  1.19442705],
        [ 0.87272193, -1.03172035, -1.06177971,  0.21152849,
         -0.64372587,  0.43193018, -0.798093  , -0.95881761,
         -0.19158293,  1.17489932, -0.363646  , -0.27585181,
         -0.78170979,  1.01652793,  1.23692196],
        [-1.96187421,  1.01567674, -0.66181262, -0.41316559,
         -0.52595315,  1.00815526, -0.23458381, -0.55713925,
         -1.0153033 ,  1.36472069, -0.91984139, -0.64914404,
         -2.07309423,  0.73989059,  0.50342281],
        [ 0.32409042, -0.00802181, -0.66181262, -1.26131421,
         -0.78483092,  1.22282735,  0.18804808, -0.71382551,
         -1.53948899,  1.96130213, -0.22459715, -1.97166509,
         -0.78170979,  0.3525983 ,  0.77779214],
        [-0.86461119, -0.69048751, -0.62181592, -1.25152788,
         -0

In [73]:
model = load_model('../items/final_model.h5', custom_objects={'asymmetric_loss': asymmetric_loss})
RUL_predicted = model.predict(preprocessed_data)
print("Predicted RUL:", RUL_predicted)

1/1 [==============================] - 1s 770ms/step
Predicted RUL: [[121.89626]]


## test window size 15

In [74]:
window_15_data = pd.read_csv('../input_data_window_15/engine_1_cycle_1_to_15.csv',header=None)

In [75]:
window_15_data

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,521.66,2388.02,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,522.28,2388.07,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,522.42,2388.03,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,522.86,2388.08,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,522.19,2388.04,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044
5,1,6,-0.0043,-0.0001,100.0,518.67,642.10,1584.47,1398.37,14.62,...,521.68,2388.03,8132.85,8.4108,0.03,391,2388,100.0,38.98,23.3669
6,1,7,0.0010,0.0001,100.0,518.67,642.48,1592.32,1397.77,14.62,...,522.32,2388.03,8132.32,8.3974,0.03,392,2388,100.0,39.10,23.3774
7,1,8,-0.0034,0.0003,100.0,518.67,642.56,1582.96,1400.97,14.62,...,522.47,2388.03,8131.07,8.4076,0.03,391,2388,100.0,38.97,23.3106
8,1,9,0.0008,0.0001,100.0,518.67,642.12,1590.98,1394.80,14.62,...,521.79,2388.05,8125.69,8.3728,0.03,392,2388,100.0,39.05,23.4066
9,1,10,-0.0033,0.0001,100.0,518.67,641.71,1591.24,1400.46,14.62,...,521.79,2388.06,8129.38,8.4286,0.03,393,2388,100.0,38.95,23.4694


In [76]:
window_size=15
preprocessed_data = preprocess_test_data(window_15_data, SCALER_PATH, window_size)

c:\Users\usatoleb\miniconda3\envs\action_project\lib\site-packages\sklearn\base.py:347: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.4.0 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
DEBUG:root:Scaled test DataFrame head: 
   oper_set_1  oper_set_2  temp_lpc_outlet  temp_hpc_outlet  temp_lpt_outlet  \
0   -0.315980   -1.372953        -1.721725        -0.134255        -0.925936   
1    0.872722   -1.031720        -1.061780         0.211528        -0.643726   
2   -1.961874    1.015677        -0.661813        -0.413166        -0.525953   
3    0.324090   -0.008022        -0.661813        -1.261314        -0.784831   
4   -0.864611   -0.690488        -0.621816        -1.251528        -0.301518   

   px_hpc_outlet  phys_fan_speed  phys_core_speed  stat_px_

In [77]:
preprocessed_data.shape

(1, 15, 15)

In [78]:
preprocessed_data

array([[[-0.31597968, -1.3729532 , -1.72172539, -0.13425518,
         -0.92593596,  1.12114057, -0.51633841, -0.86281332,
         -0.2664666 ,  0.33426184, -1.05889024, -0.6038157 ,
         -0.78170979,  1.34849274,  1.19442705],
        [ 0.87272193, -1.03172035, -1.06177971,  0.21152849,
         -0.64372587,  0.43193018, -0.798093  , -0.95881761,
         -0.19158293,  1.17489932, -0.363646  , -0.27585181,
         -0.78170979,  1.01652793,  1.23692196],
        [-1.96187421,  1.01567674, -0.66181262, -0.41316559,
         -0.52595315,  1.00815526, -0.23458381, -0.55713925,
         -1.0153033 ,  1.36472069, -0.91984139, -0.64914404,
         -2.07309423,  0.73989059,  0.50342281],
        [ 0.32409042, -0.00802181, -0.66181262, -1.26131421,
         -0.78483092,  1.22282735,  0.18804808, -0.71382551,
         -1.53948899,  1.96130213, -0.22459715, -1.97166509,
         -0.78170979,  0.3525983 ,  0.77779214],
        [-0.86461119, -0.69048751, -0.62181592, -1.25152788,
         -0

In [79]:
model = load_model('../items/window_15_model.h5', custom_objects={'asymmetric_loss': asymmetric_loss})
RUL_predicted = model.predict(preprocessed_data)
print("Predicted RUL:", RUL_predicted)

1/1 [==============================] - 1s 664ms/step
Predicted RUL: [[115.68007]]
